# Exercise E7-4: Multiple Linear Regression

We analyse the effect of various advertising media on the product sells.

Features
- **TV:** advertising dollars spent on TV for a single product in a given market (in thousands of dollars)
- **Radio:** advertising dollars spent on Radio
- **Newspaper:** advertising dollars spent on Newspaper

Output
- **Sales:** sales of a single product in a given market (in thousands of items)

Model
- Because the response variable is continuous, this is a **regression** problem.
- There are 200 **observations** (represented by the rows), and each observation is a single market.

## Step 1: Problem Analysis and Framing

In [ ]:
# import
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LinearRegression
import sklearn.metrics as sm
from sklearn.metrics import r2_score

## Step 2: Data Preparation

In [ ]:
# read CSV file from the 'data' subdirectory using a relative path
data = pd.read_csv('../data/advertising.csv', index_col=0)

In [ ]:
# check the shape of the DataFrame (rows, columns)
data.shape

In [ ]:
data.describe()

In [ ]:
# display the first 5 rows
data.head()

In [ ]:
# display the last 5 rows
data.tail()

In [ ]:
# visualise the features and the response using scatterplots
sns.pairplot(data, x_vars=['TV', 'Radio','Newspaper'], y_vars='Sales', height=5, aspect=0.8)

## Step 3: Train a Model

### Multiple Linear Regression

$y = \beta_0 + \beta_1x_1 + \beta_2x_2 + ... + \beta_nx_n$

- $y$ is the response
- $\beta_0$ is the intercept
- $\beta_1$ is the coefficient for $x_1$ (the first feature)
- $\beta_n$ is the coefficient for $x_n$ (the nth feature)

In this case:

$y = \beta_0 + \beta_1 \times TV + \beta_2 \times Radio + \beta_3 \times Newspaper$

The $\beta$ values are called the **model coefficients**. These values are "learned" during the model fitting step using the "least squares" criterion. Then, the fitted model can be used to make predictions!

### Preparing X and y 

- scikit-learn expects X (feature matrix) and y (response vector) to be NumPy arrays.
- However, pandas is built on top of NumPy.
- Thus, X can be a pandas DataFrame and y can be a pandas Series!

In [ ]:
# create a Python list of feature names
feature_cols = ['TV', 'Radio', 'Newspaper']

# use the list to select a subset of the original DataFrame
X = data[feature_cols]

# print the first 5 rows
X.head()

In [ ]:
# check the type and shape of X
print(type(X))
print(X.shape)

In [ ]:
# select a Series from the DataFrame for y
y = data['Sales']

# equivalent command that works if there are no spaces in the column name
y = data.Sales

# print the first 5 values
y.head()

In [ ]:
# check the type and shape of y
print(type(y))
print(y.shape)

### Splitting X and y into Training and Testing Sets

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
# default split 75:25
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

### Linear Regression by scikit-learn

In [ ]:
# create a model
linreg = LinearRegression()

# fit the model to our training data
linreg.fit(X_train, y_train)

### Interpret Model Coefficients

In [ ]:
# the intercept and coefficients are stored in system variables
print('b0 =', linreg.intercept_)
print('bi =', linreg.coef_)

In [ ]:
# pair the feature names with the coefficients
list(zip(feature_cols, linreg.coef_))

The result of the model fitting shows how the sales depend on the advertising:
$$y = 2.88 + 0.0466 \times TV + 0.179 \times Radio + 0.00345 \times Newspaper$$

Notes:
- This is a statement of **association**, not **causation**.
- If an increase in any ad spending was associated with a **decrease** in sales, $\beta_i$ would be **negative**.

## Step 4: Testing the Model

In [ ]:
# make predictions on the testing set
y_predicted = linreg.predict(X_test)

In [ ]:
y_predicted

In [ ]:
y_test

## Step 5: Model Evaluation Metrics

Instead, we need evaluation metrics designed for comparing continuous values.

The **three common evaluation metrics** for regression problems are:

**Mean Absolute Error** (MAE) is the mean of the absolute value of the errors:

$$\frac 1n\sum_{i=1}^n|y_i-\hat{y}_i|$$

In [ ]:
# calculate MAE using scikit-learn
from sklearn import metrics
print(metrics.mean_absolute_error(y_test, y_predicted))

**Mean Squared Error** (MSE) is the mean of the squared errors:

$$\frac 1n\sum_{i=1}^n(y_i-\hat{y}_i)^2$$

In [ ]:
# calculate MSE using scikit-learn
print(metrics.mean_squared_error(y_test, y_predicted))

**Root Mean Squared Error** (RMSE) is the square root of the mean of the squared errors:

$$\sqrt{\frac 1n\sum_{i=1}^n(y_i-\hat{y}_i)^2}$$

In [ ]:
# calculate RMSE using scikit-learn
print(np.sqrt(metrics.mean_squared_error(y_test, y_predicted)))

Comparing these metrics:

- **MAE** is the easiest to understand, because it's the average error.
- **MSE** is more popular than MAE, because MSE "punishes" larger errors.
- **RMSE** is even more popular than MSE, because RMSE is interpretable in the "y" units.

### R-squared

In [ ]:
# Explained variance (1 would be the best prediction)
eV = round(sm.explained_variance_score(y_test, y_predicted), 2)
print('Explained variance score ',eV )

In [ ]:
# R-squared
r2_score(y_test, y_predicted)

In [ ]:
# Visualise the Polymonial Regression results
plt.title('Multiple Linear Regression')
plt.scatter(y_test, y_predicted, color='blue')
plt.show()

### Can We Improve the Model by Feature Selection?

**Newspaper** has very low coeficient, we can try to remove it and evaluate the new regression again.

In [ ]:
# create a Python list of feature names
feature_cols = ['TV', 'Radio']

# use the list to select a subset of the original DataFrame
X = data[feature_cols]

# select a Series from the DataFrame
y = data.Sales

# split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# fit the model to the training data (learn the coefficients)
linreg.fit(X_train, y_train)

# make predictions on the testing set
y_predicted = linreg.predict(X_test)

# compute the RMSE of our predictions
print(np.sqrt(metrics.mean_squared_error(y_test, y_predicted)))

The RMSE **decreased** when we removed Newspaper from the model. (Error is something we want to minimize, so **a lower number for RMSE is better**.)

In [ ]:
# R-squared
r2_score(y_test, y_predicted)

We are aiming at bigger R-squared, close to 1.00, so improvement is valued.

## Exercise
Apply Multiple Regression Analysis to the diabetes data (diabetes.cv).

## Resources

- [An Introduction to Statistical Learning](https://www.statlearning.com/) and [related videos](https://www.dataschool.io/15-hours-of-expert-machine-learning-videos/) 
- [Introduction to linear regression](http://people.duke.edu/~rnau/regintro.htm)
-  https://scikit-learn.org/stable/auto_examples/linear_model/plot_ols.html#sphx-glr-auto-examples-linear-model-plot-ols-py
